# Displaying hyperspectral images in extra channels

Before you start, make sure that you have the correct kernel. Check if the kernel is "Python 2" (there is a legend in the upper right corner, below the "Logout" button). If it is not Python 2, select Kernel → Change Kernel → Python 2 in the menu bar.
Make sure that you fill any place that says "YOUR CODE HERE" or "YOUR ANSWER HERE", and that you erase the line "raise NotImplementedError()" in the code cells.

# Introduction

Now that we know a way to include an extra channel on an RGB image, let's use these tools for displaying information
from our hyperspectral images.

Import modules:

In [ ]:
import scipy.io
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as img
import pandas as pd
from scipy import interpolate
import os
import imageio
import sys
from scipy import signal
sys.path.append('../tools/')
from Tools import wavelengthToRGB as wl2rgb
from Tools import *
%load_ext autoreload
%autoreload 2
%pylab inline


Hyperspectral image functions: 

In [ ]:
def load_PARC_image(file_name):
    # This function reads a .bin file from the PARC hyperspectral camera
    # and loads it into python as a 3-D matrix (numpy array)
    my_img = np.fromfile(file_name, dtype = np.uint16)
    my_img = np.reshape(my_img, (141,488,648))
    return my_img

def get_waves(my_img, resolution = 5, init_nm = 400):
    # given a PARC hyperspectral camera image as input
    # returns the range of wavelengths to which each slice/matrix corresponds. 
    waves = np.arange(init_nm, init_nm+resolution*my_img.shape[0], resolution)
    return waves

def get_nm_ind(waves, nm):
    # Returns the wavelength available in a PARC hyperspectral camera image
    # and its corresponding index closest
    # To a wavelength of interest. 
    # ind_nm, nm = get_nm_ind(waves, nm)
    return [(i,waves[i]) for i in range(len(waves)) if 
            np.abs(waves[i]-nm)==np.min(np.abs(waves-nm))][0]

def display_full_single_channel(my_img, ind_nm, nm, save = False, saveDir = 'image_dir'):
    # This function displays a single channel/wavelength/slice
    # in grayscale
    # ind_nm and nm can be obtained from 
    # ind_nm, nm = get_nm_ind(waves, nm)
    plt.figure(figsize = (10,10))
    single_slice = my_img[ind_nm,:,:]
    plt.imshow(single_slice, cmap = "gray")
    plt.title('Single Channel: ' + str(int(nm))+"nm", fontsize = 24)

    if save: 
        path_to_file = os.path.join(saveDir, 'single_channel_'+str(nm)+'_nm.pkl')
        pkl.dump(single_slice, open(path_to_file, "wb"))
    
    
def draw_box(ax, rlim, clim):
    # This function is used to draw a red box around a region 
    # that we'll zoom into. 
    
    xbox = np.linspace(clim[0], clim[1])
    ybox = np.linspace(rlim[0], rlim[1])
    ax.plot( xbox , rlim[0]*np.ones(len(xbox)),'r' )
    ax.plot( xbox , rlim[1]*np.ones(len(xbox)),'r' )
    ax.plot( clim[0]*np.ones(len(ybox)), ybox ,'r' )
    ax.plot( clim[1]*np.ones(len(ybox)), ybox ,'r' )
    
def display_part_single_channel(my_img, ind_nm, nm, rlim, clim, fix_tics = True):
    # This function displays in grayscale a single channel 
    # corresponding to wavelength nm (and index ind_nm)
    # it zooms in to a region specified by the coordinate range rlim and clim
    
    fig, (ax1,ax2) = plt.subplots(1,2, figsize = (14,8))
    ax1.imshow(my_img[ind_nm,:,:], cmap = "gray")
    draw_box(ax1, rlim, clim)
    ax1.set_xlim(0,my_img.shape[2])
    ax1.set_ylim(my_img.shape[1],0)

    ax2.imshow(my_img[ind_nm,rlim[0]:rlim[1],clim[0]:clim[1]], cmap = "gray")
    if fix_tics:
        a=ax2.get_yticks().tolist()
        new_tics = [ int(yt) for yt in np.linspace(rlim[0], rlim[1], len(a))]
        tkn = ax2.set_yticklabels(new_tics)

        old_tics=ax2.get_xticks().tolist()
        new_tics = [ int(xt) for xt in np.linspace(clim[0], clim[1], len(old_tics))]
        tkn = ax2.set_xticklabels(new_tics)
        
def display_single_channel_centered_at(my_img, ind_nm, nm, rcenter, ccenter, wd=20,fix_tics = True):
    # This function displays in grayscale a single channel 
    # corresponding to wavelength nm (and index ind_nm)
    # it zooms in to a region specified by the pixel with coordinates 
    # rcenter and ccenter. 
    
    fig, (ax1,ax2) = plt.subplots(1,2, figsize = (14,8))
    ax1.imshow(my_img[ind_nm,:,:], cmap = "gray")
    draw_box(ax1, [rcenter-wd, rcenter+wd], [ccenter-wd, ccenter+wd])
    ax1.set_xlim(0,my_img.shape[2])
    ax1.set_ylim(my_img.shape[1],0)
    
    ax2.imshow(my_img[ind_nm,rcenter-wd:rcenter+wd,ccenter-wd:ccenter+wd], cmap = "gray")
    if fix_tics:
        a=ax2.get_yticks().tolist()
        new_tics = [ int(yt) for yt in np.linspace(rcenter-wd, rcenter+wd, len(a))]
        tkn = ax2.set_yticklabels(new_tics)

        old_tics=ax2.get_xticks().tolist()
        new_tics = [ int(xt) for xt in np.linspace(ccenter-wd, ccenter+wd, len(old_tics))]
        tkn = ax2.set_xticklabels(new_tics)
        
def get_max_wavelength(spectrum, waves):
    # This function finds the wavelength with the 
    # highest intensity in a spectrum
    
    ind_max = [i for i in range(len(spectrum)) if spectrum[i] == np.max(spectrum)][0]
    return int(waves[ind_max])
        
def single_pxl_spectrum(my_img, ind_nm, nm, rcenter, ccenter, wd = 40):
    
    #This function plots the full spectrum for a single pixel
    #with coordinates rcenter, ccenter
    
    display_single_channel_centered_at(my_img, ind_nm, nm, rcenter, ccenter, wd)
    
    plt.figure(figsize = (6,6)) 

    waves = get_waves(my_img)
    spectrum = my_img[:, rcenter, ccenter]
    max_freq = get_max_wavelength(spectrum, waves)
    
    plt.plot(waves, spectrum, lw = 3)
    plt.xlabel('Wavelength (nm)', fontsize = 24)
    plt.ylabel('Intensity (units)', fontsize = 24)

    plt.title('Max. Frequency = ' + str(max_freq) + ' nm', fontsize = 24)
    
def hyperspectral_2_RGB(my_img, waves):
    ### This function takes a hyperspectral image
    ### and "converts" it to an RGB representation
    ### it uses some of the functions in the Tools.py library
    ### written by Teresa Tamayo. 
    my_img_rolled = np.rollaxis(np.rollaxis(my_img,1,0),2,1)
    rgb_image = eye_sensitivity(my_img_rolled,waves,tranformation='linear')
    rgb_image *=1.5
    max_pixel = np.max(rgb_image)
    rgb_image_final = rgb_image/max_pixel**1
    return rgb_image_final

Function definitions:

In [ ]:
def get_uniform_color_img(rgb_vec, image):
    #First make a 3-D matrix of the correct shape (same shape as your image)
    uniform_color = np.zeros(image.shape)

    #Now create three equivalent matrices, R, G, B 
    single_channel_size = image.shape[:2]
    for i in range(len(rgb_vec)):
        uniform_color[:,:,i] = rgb_vec[i]*np.ones(single_channel_size)
        
    return uniform_color
    
def get_extra_channel_color(uniform_color_img, extra_channel):
    extra_channel_color = np.zeros(uniform_color_img.shape)
    for i in range(3):
        extra_channel_color[:,:,i] = np.multiply(uniform_color_img[:,:,i], extra_channel)
    #This normalization is important:
    extra_channel_color = extra_channel_color / np.float(extra_channel_color.max())
    return extra_channel_color

In [ ]:
Copy and paste below your functions:
    * image_interpolation_t
    * make_interpolation_movie

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Load a RGB image and display it:
Note: It is in image_dir

In [ ]:
my_img = img.imread('image_dir/LEDs_4_RGB.png')
# YOUR CODE HERE
raise NotImplementedError()

Load hyperspectral and get a cube file (a "3D-matrix"), called it hyper_img.
Hint: Check out the functions written at the begining.

In [ ]:
file_name = 'image_dir/LEDs_4.bin'
# YOUR CODE HERE
raise NotImplementedError()

Now obtain in an array the wavelengths sampled in the image:

In [ ]:
waves = get_waves(hyper_img)

As a warm up, get a the fake RGB image associated with this hyperspectral information, and display it.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Now let's get an extra_channel, let's say the image at 520nm.
**Plot a single wavelenght in gray scale**

In [ ]:
nm = 520
ind_nm, nm = get_nm_ind(waves, nm)
extra_channel = hyper_img[ind_nm, :,:]
# YOUR CODE HERE
raise NotImplementedError()

With this extra channel, let's make movie: 

In [ ]:
num_cycles = 5
cycle = signal.triang(20, sym = 0)
t_vec = np.array(num_cycles * list(cycle) )

In [ ]:
#Input Parameters 
my_fps = 10
wavelength = 520

# Get the extra channel image
# YOUR CODE HERE
raise NotImplementedError()

# Get the RGB vector that wavelengt
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Call the necessary function add some color to the extra channel with
# the rgb color of that wavelength
# YOUR CODE HERE
raise NotImplementedError()

# Make a movie
# YOUR CODE HERE
raise NotImplementedError()

Now let's put everything together, we are getting ready to process any hyperspectral image!!


In [ ]:
def make_interpolation_movie_hyper(hyper_file,wavelength, movie='movie.gif',num_cycles = 5,fps=10):
    ''' This creates a gif file that contains an extrachannel that varies on time
    with the information of the 
     intensities and color of a given wavelenght to the fake RGB image of 
     a hyperspectral information
    INPUTS:
        hyper_file -> name with path of a cube file
        wavelength -> wavelenght to display
        movie -> name of the gif file
        num_cycles -> number of cycles for the movie
        '''
    # YOUR CODE HERE
    raise NotImplementedError()
    

Test your function with different wavelenghts:

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Futher directions:

## How would you add another channel? Let's say two different wavelengths?
## How would you add a band?